In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.path.append('/kaggle/input/catechol-benchmark-hackathon/')

from utils import INPUT_LABELS_FULL_SOLVENT, INPUT_LABELS_SINGLE_SOLVENT, INPUT_LABELS_NUMERIC, INPUT_LABELS_SINGLE_FEATURES, INPUT_LABELS_FULL_FEATURES, load_data, load_features, generate_leave_one_out_splits, generate_leave_one_ramp_out_splits

In the cells below we create the base classes of the two main objects you must write for the competition. 

The first thing to write is a SmilesFeaturizer, which will take the solvent molecules and create a machine-learning ready featurization of the molecule. Finding better ways of featurizing solvents is one of the goals of the hackathon, however, you can also skip this step and use the pre-computed featurizations given in the utils file. Further down, you can see a SmilesFeaturizer that loads all the precomputed representations. A **featurizer** object simply consists of:
- An initialization function
- A featurize function that takes 

The second one being a **model** which has:
- An initialization function, where the model internally defines which featurizer to use
- A "train_model" which lets the model train on data given by X_train, y_train as pandas data-frames. 
- A "predict" which takes a data frame of test inputs and makes a prediction

In [ ]:
from abc import ABC, abstractmethod

class SmilesFeaturizer(ABC):
    def __init__(self):
        raise NotImplementedError

    def featurize(X, Y):
        raise NotImplementedError

class BaseModel(ABC):
    def __init__(self):
        pass

    def train_model(self, X_train, y_train):
        raise NotImplementedError

    def predict(self):
        raise NotImplementedError

In the next cell we define two featurizers, which allow you to use the pre-computed featurizations from the original benchmark paper. These are:

- drfps
- fragprints
- acs_pca_descriptors
- spange_descriptors

You can refer to the paper for more details on them. We also include the simple SMILES string featurization which can be chained into more complicated representations.

The first featurizer simply uses the features directly. The second one is expanded to featurize *mixed* solvents too, which is done by taking a weighted average of the two single-solvent features.

We also show how to write code for a simple multi-layer perceptron on the data.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

torch.set_default_dtype(torch.double) 

class PrecomputedFeaturizer(SmilesFeaturizer):
    def __init__(self, features = 'spange_descriptors'):
        assert features in ['drfps_catechol', 'fragprints', 'smiles', 'acs_pca_descriptors', 'spange_descriptors']
        self.features = features

        #load the featurizer
        self.featurizer = load_features(self.features)

        #load the dimension, which depends on the featurizer + the 2 numeric dimensions
        self.feats_dim = self.featurizer.shape[1] + 2
        

    def featurize(self, X):
        # obtain the numeric inputs i.e. residence time and temperature
        X_numeric = X[INPUT_LABELS_NUMERIC]
        # obtain the relevant molecule information
        X_smiles = X[INPUT_LABELS_SINGLE_FEATURES]
        
        X_smiles_feat = self.featurizer.loc[X["SOLVENT NAME"]]

        X_numeric_tensor = torch.tensor(X_numeric.values)
        X_smiles_feat_tensor = torch.tensor(X_smiles_feat.values)

        X_out = torch.cat((X_numeric_tensor, X_smiles_feat_tensor), dim=1)

        return X_out
        

class PrecomputedFeaturizerMixed(SmilesFeaturizer):
    def __init__(self, features = 'spange_descriptors'):
        assert features in ['drfps_catechol', 'fragprints', 'smiles', 'acs_pca_descriptors', 'spange_descriptors']
        self.features = features

        #load the featurizer
        self.featurizer = load_features(self.features)

        #load the dimension, which depends on the featurizer + the 2 numeric dimensions
        self.feats_dim = self.featurizer.shape[1] + 2

    def featurize(self, X):
        # obtain the numeric inputs i.e. residence time and temperature
        X_numeric = X[INPUT_LABELS_NUMERIC]
        # obtain the relevant molecule information
        X_smiles = X[INPUT_LABELS_FULL_FEATURES]

        X_smiles_A_feat = self.featurizer.loc[X["SOLVENT A NAME"]]
        X_smiles_B_feat = self.featurizer.loc[X["SOLVENT B NAME"]]

        X_numeric_tensor = torch.tensor(X_numeric.values)
        # take the weighted average accoriding to the SolventB% in the mixture
        X_smiles_feat_tensor = X_smiles_A_feat.values * (1 - X["SolventB%"].values.reshape(-1, 1)) + X_smiles_B_feat.values * X["SolventB%"].values.reshape(-1, 1)
        X_smiles_feat_tensor = torch.tensor(X_smiles_feat_tensor)

        X_out = torch.cat((X_numeric_tensor, X_smiles_feat_tensor), dim=1)

        return X_out
        

class MLPModel(nn.Module, BaseModel):
    def __init__(self, features = 'spange_descriptors', hidden_dims = [64, 64], output_dim = 3, dropout=0.0, data = 'single'):
        super(MLPModel, self).__init__()
        layers = []

        if data == 'single':
            self.smiles_featurizer = PrecomputedFeaturizer(features = features)
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed(features = features)
        
        input_dim = self.smiles_featurizer.feats_dim
        prev_dim = input_dim

        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_dim = h_dim

        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def train_model(
        self,
        train_X,
        train_Y,
        criterion=nn.MSELoss,
        optimizer=torch.optim.Adam,
        num_epochs=100,
        batch_size=1048,
        device="cpu",
        verbose=True,
        lr = 1e-3,
    ):
        """
        Trains the MLP model.
        Args:
            train_X: pandas array of training inputs
            train_Y: pandas array of training outputs
            criterion: Loss function.
            optimizer: Optimizer.
            num_epochs (int): Number of epochs.
            batch_size (int): Batch size.
            device (torch.device): Device to train on.
            verbose (bool): Whether to print progress.
            lr (float): learning rate to use
        """
        self.train()
        
        train_X_tensor = self.smiles_featurizer.featurize(train_X)
        train_Y_tensor = torch.tensor(train_Y.values)
        
        train_dataset = TensorDataset(train_X_tensor, train_Y_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = False)
        
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device)

        criterion = criterion()
        optimizer = optimizer(self.parameters(), lr = lr)

        for epoch in range(1, num_epochs + 1):
            self.train()
            running_loss = 0.0
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(train_loader.dataset)

            if (verbose) and (epoch % 10 == 1):
                msg = f"Epoch {epoch}/{num_epochs} | Train Loss: {epoch_loss:.4f}"

                # if val_loader is not None:
                #     val_loss = self.evaluate(val_loader, criterion, device)
                #     msg += f" | Val Loss: {val_loss:.4f}"

                # print(msg)

    def predict(self, X):
        X = self.smiles_featurizer.featurize(X)
        return self.forward(X)


    def forward(self, x):
        return self.model(x)

From this point onward the cross-validation procedure is calculated. **For a submission to be valid the next three cells must be the final three of your submission, and you can only modify the lines where the models are defined.**

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MLPModel() # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MLPModel(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################